In [2]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.solvers import PreconditionedRichardson as PreRic

from xfem import *
from xfem.mlset import *

from ngsolve.webgui import *
DrawDC = MakeDiscontinuousDraw(Draw)

In [3]:
geo = SplineGeometry()
geo.AddRectangle((-0.2,-0.2),(1.2,1.2), bcs=("bottom","right","top","left"))
mesh = Mesh(geo.GenerateMesh(maxh=0.1))

In [5]:
level_sets = (-y, x-1, y-1, -x) # 这样写水平集的顺序可以和上面的边界名称顺序是一致的
level_sets_p1 = tuple(GridFunction(H1(mesh,order=1)) for i in range(len(level_sets)))
for i, lsetp1 in enumerate(level_sets_p1):
    InterpolateToP1(level_sets[i],lsetp1)
    DrawDC(lsetp1,-1,1,mesh)

square = DomainTypeArray((NEG,NEG,NEG,NEG))
boundary = square.Boundary()


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [13]:
V = H1(mesh,order=3,dgjumps=True)
gfu = GridFunction(V)

# Multiple level set cut information

In [9]:
mlci = MultiLevelsetCutInfo(mesh,level_sets_p1)

els_hasneg = mlci.GetElementsWithContribution(square)
els_if = mlci.GetElementsWithContribution(boundary)

Draw(BitArrayCF(els_hasneg),mesh)
Draw(BitArrayCF(els_if),mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [14]:
facets_gp = GetFacetsWithNeighborTypes(mesh,a=els_hasneg,b=els_if,use_and=True)
freedofs = GetDofsOfElements(V,els_hasneg) & V.FreeDofs()

In [12]:
els_if_single = {}
for i,dtt in enumerate(boundary):
    els_if_single[dtt] = mlci.GetElementsWithContribution(dtt)
    Draw(BitArrayCF(els_if_single[dtt]),mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [24]:
gamma_n = 100 # Nitsche parameter
gamma_s = 0.5  # penalty parameter

u,v = V.TnT()
h = specialcf.mesh_size
normals = square.GetOuterNormals(level_sets_p1)

In [25]:
dx = dCut(level_sets_p1,square,definedonelements=els_hasneg)
ds = {dtt:dCut(level_sets_p1,dtt,definedonelements=els_if_single[dtt]) for dtt in boundary}
dw = dFacetPatch(definedonelements=facets_gp)

In [26]:
a = RestrictedBilinearForm(V, elements_restriction=els_hasneg, facet_restriction=facets_gp, check_unused=False)
a += InnerProduct(Grad(u),Grad(v)) * dx
a += gamma_s / (h**2) * (u - u.Other()) * (v - v.Other()) * dw

for bnd, n in normals.items():
    a += -InnerProduct(Grad(u)*n, v) * ds[bnd]
    a += -InnerProduct(Grad(v)*n, u) * ds[bnd]
    a += gamma_n / h * InnerProduct(u, v) * ds[bnd]
    

In [32]:
coef_f = 32 *(y * (1-y) + x*(1-x)) 
f = LinearForm(V)
f += coef_f * v * dx

In [33]:
f.Assemble()
a.Assemble()
inv = a.mat.Inverse(freedofs=freedofs, inverse="sparsecholesky")
gfu.vec.data = PreRic(a=a,rhs=f.vec,pre=inv,freedofs=freedofs)

it = 0  ||res||_2 = 1.0726061739096793
it = 1  ||res||_2 = 1.0779929201840319e-14


In [34]:
DrawDC(square.Indicator(level_sets_p1),0,gfu,mesh)
DrawDC(square.Indicator(level_sets_p1),0,exact_u,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [35]:
exact_u = 16 *y * (1-y) * x*(1-x)
err_l2 = sqrt(Integrate((exact_u-gfu)**2*dx.order(6), mesh)) 
print(err_l2)

3.5396155442718646e-06
